# Deep RL hands-on by Maxim Lapan
## Chapter 4 The Corss-Entropy Method

* conda activate gym 
 
### Method step

1. Play N number of episodes using our current model and environment.
1. Calculate the total reward for every episode and decide on a reward boundary. Usually, we use some percentile of all rewards, such as 50th or 70th.
1. Throw away all episodes with a reward below the boundary.
1. Train on the remaining "elite" episodes using obervations as the input and issued actions as the desired outtput.
1. Repeat from step 1 until we become satisfied with the result.

## Cart pole

In [4]:
import gym
from collections import namedtuple
import numpy as np
import tensorflow as tf

import torch
import torch.nn as nn
import torch.optim as optim
import datetime

In [5]:
# set hyperparameters
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [6]:
# set basic neural net module with FC layer and ReLU
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


In [7]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [8]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=0) # below the input obs has no [], I use dim=0 instead of dim=1 in the original code
    while True:
        obs_v = torch.FloatTensor(obs) # original code use [obs], but I deleted it. 
        # If you use [obs], the output data will be also embraced with [], like [[0.9, 0.1]]
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy() # I deleted [0], there is no additional dimension in the output.
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [9]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue # forward to next iter.
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [10]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="recording1_cartpole_cross", force=True) # if you wanna record the video
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [11]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convenient to record the date with current time
LOGDIR = './tmp/cross_entroy/cartpole/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR) # tf 2.0 has changed the modules of tensorboard
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v.item(), iter_no)
        tf.summary.scalar("reward_bound", reward_b, iter_no)
        tf.summary.scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break

0: loss=0.694, reward_mean=28.9, reward_bound=27.5
1: loss=0.684, reward_mean=24.3, reward_bound=30.5
2: loss=0.668, reward_mean=25.1, reward_bound=31.5
3: loss=0.672, reward_mean=26.6, reward_bound=24.5
4: loss=0.646, reward_mean=38.5, reward_bound=32.0
5: loss=0.636, reward_mean=42.3, reward_bound=52.0
6: loss=0.621, reward_mean=45.1, reward_bound=55.5
7: loss=0.630, reward_mean=50.1, reward_bound=58.0
8: loss=0.611, reward_mean=58.2, reward_bound=71.0
9: loss=0.611, reward_mean=43.9, reward_bound=48.0
10: loss=0.602, reward_mean=47.8, reward_bound=43.5
11: loss=0.594, reward_mean=72.3, reward_bound=99.0
12: loss=0.583, reward_mean=46.4, reward_bound=44.0
13: loss=0.571, reward_mean=58.4, reward_bound=69.0
14: loss=0.577, reward_mean=74.5, reward_bound=81.5
15: loss=0.573, reward_mean=58.6, reward_bound=59.0
16: loss=0.574, reward_mean=56.9, reward_bound=59.5
17: loss=0.553, reward_mean=65.2, reward_bound=72.0
18: loss=0.565, reward_mean=65.1, reward_bound=80.5
19: loss=0.551, reward

## Frozen Lake

In [12]:
e = gym.make("FrozenLake-v0")
print(e.observation_space)
print(e.action_space)
e.reset()
e.render()

Discrete(16)
Discrete(4)

SFFF
FHFH
FFFH
HFFG


In [13]:
# use same hyperparameter, Net module, Episode tuple of cart pole agent
# use same function of iterate batch and filter batch too.
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low) # return 1d array with 16 elements of zero
        res[observation] = 1.0 # make 1.0 at the observed space in the frozen lake. 
        return res
# because the observation space of lake is discrete, not like cart pole. 

In [14]:
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [15]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convenient to record the date with current time
LOGDIR = './tmp/cross_entroy/frozenlake/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR) # tf 2.0 has changed the modules of tensorboard
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    if iter_no%50 == 0: # prevent printing too many iter result 
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v.item(), iter_no)
        tf.summary.scalar("reward_bound", reward_b, iter_no)
        tf.summary.scalar("reward_mean", reward_m, iter_no)
    if reward_m > 0.8:
        print("Solved!")
        break
    if iter_no > 500: 
        # this naive agent can't solve the problem. percentile selection is not comparable with the frozen lake returns.
        break

0: loss=1.385, reward_mean=0.0, reward_bound=0.0
50: loss=1.040, reward_mean=0.0, reward_bound=0.0
100: loss=1.122, reward_mean=0.1, reward_bound=0.0
150: loss=0.492, reward_mean=0.0, reward_bound=0.0
200: loss=0.026, reward_mean=0.0, reward_bound=0.0
250: loss=0.034, reward_mean=0.0, reward_bound=0.0
300: loss=0.001, reward_mean=0.0, reward_bound=0.0
350: loss=0.000, reward_mean=0.0, reward_bound=0.0
400: loss=0.000, reward_mean=0.0, reward_bound=0.0
450: loss=0.000, reward_mean=0.0, reward_bound=0.0
500: loss=0.000, reward_mean=0.0, reward_bound=0.0


## Frozen lake, tweaked agent

In [16]:
import random

In [17]:
# set hyperparameter
HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9

In [18]:
# other variables and function same with above
def filter_batch(batch, percentile):
    disc_rewards = list(map(lambda s: s.reward * (GAMMA ** len(s.steps)), batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation, example.steps))
            train_act.extend(map(lambda step: step.action, example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound

In [19]:
random.seed(12345)
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [20]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convenient to record the date with current time
LOGDIR = './tmp/cross_entroy/frozenlake_tweaked/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR) # tf 2.0 has changed the modules of tensorboard
full_batch = []
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
    full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
    if not full_batch:
        continue
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts)
    full_batch = full_batch[-500:]

    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    if iter_no%500 == 0: # prevent printing too many iter result 
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_mean, reward_bound))
    if iter_no%25 == 0: # making faster.
        with writer.as_default():
            tf.summary.scalar("loss", loss_v.item(), iter_no)
            tf.summary.scalar("reward_bound", reward_bound, iter_no)
            tf.summary.scalar("reward_mean", reward_mean, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break
    if iter_no > 1000: 
        #if the agent cann't solve the problem, break.
        break

0: loss=1.397, reward_mean=0.0, reward_bound=0.0
500: loss=1.027, reward_mean=0.1, reward_bound=0.4
1000: loss=0.904, reward_mean=0.1, reward_bound=0.4


## Frozen lake, nonslippery environment

* almost same hyperparameter, function and so on.

In [45]:
random.seed(12345)
env = gym.envs.toy_text.frozen_lake.FrozenLakeEnv(is_slippery=False)
env = gym.wrappers.TimeLimit(env, max_episode_steps=100)
env = DiscreteOneHotWrapper(env)
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [47]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # it's convenient to record the date with current time
LOGDIR = './tmp/cross_entroy/frozenlake_tweaked/' + current_time + '/' 
writer = tf.summary.create_file_writer(LOGDIR) # tf 2.0 has changed the modules of tensorboard
full_batch = []
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
    full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
    if not full_batch:
        continue
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts)
    full_batch = full_batch[-500:]

    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    if iter_no%10 == 0: # prevent printing too many iter result 
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_mean, reward_bound))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v.item(), iter_no)
        tf.summary.scalar("reward_bound", reward_bound, iter_no)
        tf.summary.scalar("reward_mean", reward_mean, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break
    if iter_no > 1000: 
        #if the agent cann't solve the problem, break.
        break

0: loss=1.348, reward_mean=0.0, reward_bound=0.0
10: loss=1.305, reward_mean=0.0, reward_bound=0.0
20: loss=1.246, reward_mean=0.1, reward_bound=0.0
30: loss=1.199, reward_mean=0.1, reward_bound=0.0
40: loss=1.087, reward_mean=0.1, reward_bound=0.2
50: loss=0.934, reward_mean=0.2, reward_bound=0.3
60: loss=0.801, reward_mean=0.2, reward_bound=0.3
70: loss=0.555, reward_mean=0.4, reward_bound=0.5
80: loss=0.605, reward_mean=0.5, reward_bound=0.4
90: loss=0.453, reward_mean=0.5, reward_bound=0.4
100: loss=0.502, reward_mean=0.7, reward_bound=0.4
110: loss=0.434, reward_mean=0.7, reward_bound=0.4
Solved!
